In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Load and clean data, setting 'Epoch' as the index
data = pd.read_csv("E:\\Vibration\\Stats\\Overlapping Epochs\\All Frequency EMdelay\\StatsGeneral_5_AF.csv")  # Replace with your actual file path
data = data.replace([float('inf'), float('-inf')], float('nan')).dropna(subset=['CVForce', 'CVISI', 'SDCST'])
data.set_index('Epoch', inplace=True)  # Set 'Epoch' as the index to ensure time alignment

# Define common ARIMA model parameters for all subjects
common_p = 1  # Adjust this value as desired
common_d = 2  # Adjust this value as desired
common_q = 1  # Adjust this value as desired

print(f"Using common ARIMA parameters: p={common_p}, d={common_d}, q={common_q}")

# Function to perform ADF, ACF, and PACF analysis
def analyze_series(series, subject_id, p, d, q):
    print(f"\nAnalysis for Subject {subject_id}")
    
    # Augmented Dickey-Fuller (ADF) Test
    adf_result = adfuller(series.dropna())  # Drop NaNs for the ADF test
    adf_stat, adf_p_value = adf_result[0], adf_result[1]
    print("ADF Statistic:", adf_stat)
    print("p-value:", adf_p_value)
    for key, value in adf_result[4].items():
        print(f"Critical Value ({key}): {value}")
    
    # Set differencing (d) based on common parameter
    if d == 0:
        print("Using common differencing: no differencing (d=0).")
    else:
        print(f"Using common differencing: applying differencing (d={common_d}).")
    
    # Determine the maximum number of lags based on series length and 50% constraint
    max_lags = min(10, (len(series.dropna()) - 1) // 2)
    
    # Plot ACF and PACF only if the series has enough data points
    if max_lags > 0:
        fig, axes = plt.subplots(1, 2, figsize=(16, 4))
        plot_acf(series.dropna(), lags=max_lags, ax=axes[0], zero=False)
        plot_pacf(series.dropna(), lags=max_lags, ax=axes[1], zero=False)
        plt.suptitle(f"ACF and PACF for Subject {subject_id} (ARIMA({p}, {d}, {q}))", fontsize=14)
        plt.show()
    
    # Calculate ACF and PACF values up to max_lags, excluding zero-lag term
    acf_values = acf(series.dropna(), nlags=max_lags)[1:]  # Remove zero-lag for ACF
    pacf_values = pacf(series.dropna(), nlags=max_lags)[1:]  # Remove zero-lag for PACF
    
    print(f"Suggested parameters for Subject {subject_id}: p={p}, d={d}, q={q}")
    
    return adf_stat, adf_p_value, acf_values, pacf_values

# Loop through each subject and analyze CVForce using common parameters
for subject_id, subject_data in data.groupby('SubjectID'):
    y = subject_data['CVForce']
    
    # Perform analysis with common ARIMA parameters
    adf_stat, adf_p_value, acf_values, pacf_values = analyze_series(y, subject_id, common_p, common_d, common_q)
    
    # Display the ADF test results and suggested (common) parameters for each subject
    print(f"\nSubject {subject_id} - ADF Statistic: {adf_stat:.3f}, p-value: {adf_p_value:.3f}")
    print(f"Using common ARIMA parameters: (p={common_p}, d={common_d}, q={common_q}) for all subjects.")
